# Temporal Hypergraph Toolkit - Quick Demo

**Find cross-vault bridges in your Obsidian notes in 2 minutes.**

This notebook demonstrates the [temporal-hypergraph toolkit](https://github.com/dabirdwell/structured-emergence/tree/main/tools/temporal-hypergraph) - a tool for analyzing knowledge graphs with:
- **Hyperedge relationships** (concepts can belong to multiple documents simultaneously)
- **Temporal versioning** (track how your thinking evolves)
- **Cross-vault bridges** (find concepts that connect separate knowledge bases)

## Setup

In [ ]:
# Clone the toolkit
!git clone https://github.com/dabirdwell/structured-emergence.git
%cd structured-emergence/tools/temporal-hypergraph

In [ ]:
# Import the toolkit
from nested_hypergraph import NestedHypergraph, HyperEdge, build_from_vault
from cross_vault_bridge import merge_hypergraphs, find_cross_vault_bridges
from visualization_export import to_cytoscape_json, save_json
print("Toolkit loaded ✓")

## Option 1: Upload Your Vault

Upload a zip of your Obsidian vault (or any folder of markdown files):

In [ ]:
from google.colab import files
import zipfile
import os

print("Upload your vault as a .zip file:")
uploaded = files.upload()

# Extract
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as z:
            z.extractall('my_vault')
        print(f"Extracted to ./my_vault")
        
# Find the vault root (first directory with .md files)
vault_path = 'my_vault'
for root, dirs, files_list in os.walk('my_vault'):
    if any(f.endswith('.md') for f in files_list):
        vault_path = root
        break
print(f"Vault path: {vault_path}")

## Option 2: Use Sample Data

Skip the upload and use generated sample data:

In [ ]:
# Create sample vault structure
import os

os.makedirs('sample_vault_a', exist_ok=True)
os.makedirs('sample_vault_b', exist_ok=True)

# Vault A: Philosophy notes
sample_a = {
    'consciousness.md': '# Consciousness\n\nRelated to [[emergence]] and [[qualia]].\nSee also [[philosophy of mind]].',
    'emergence.md': '# Emergence\n\n[[complexity]] gives rise to [[consciousness]].\nKey concept in [[systems thinking]].',
    'philosophy_of_mind.md': '# Philosophy of Mind\n\nStudies [[consciousness]], [[qualia]], and [[intentionality]].',
    'systems_thinking.md': '# Systems Thinking\n\nFramework connecting [[emergence]] and [[complexity]].',
}

# Vault B: AI research notes  
sample_b = {
    'artificial_consciousness.md': '# Artificial Consciousness\n\nCan AI achieve [[consciousness]]? Relates to [[emergence]] in neural networks.',
    'neural_networks.md': '# Neural Networks\n\nShow [[emergence]] of capabilities. Connection to [[consciousness]] debated.',
    'alignment.md': '# AI Alignment\n\nEnsuring AI goals match human values. Requires understanding [[consciousness]].',
    'complexity_science.md': '# Complexity Science\n\nStudies [[emergence]] and [[systems thinking]] in AI.',
}

for name, content in sample_a.items():
    with open(f'sample_vault_a/{name}', 'w') as f:
        f.write(content)
        
for name, content in sample_b.items():
    with open(f'sample_vault_b/{name}', 'w') as f:
        f.write(content)

print("Sample vaults created ✓")
vault_path = 'sample_vault_a'  # Use this for single-vault analysis

## Build the Hypergraph

In [ ]:
# Build hypergraph from vault
hg = build_from_vault(vault_path, 'my_vault')

stats = hg.stats()
print(f"📊 Vault Analysis:")
print(f"   Documents: {stats['total_hyperedges']}")
print(f"   Unique concepts: {stats['total_concepts']}")
print(f"   Max nesting depth: {stats['max_nesting_depth']}")

## Cross-Vault Bridge Analysis

Find concepts that bridge multiple knowledge bases:

In [ ]:
# Build both sample vaults
vault_a = build_from_vault('sample_vault_a', 'Philosophy')
vault_b = build_from_vault('sample_vault_b', 'AI_Research')

print(f"Philosophy vault: {vault_a.stats()['total_hyperedges']} docs, {vault_a.stats()['total_concepts']} concepts")
print(f"AI Research vault: {vault_b.stats()['total_hyperedges']} docs, {vault_b.stats()['total_concepts']} concepts")

# Merge and find bridges
merged = merge_hypergraphs(vault_a, vault_b, name='combined')
bridges = find_cross_vault_bridges(merged)

print(f"\n🌉 Found {len(bridges)} bridge concepts:\n")
for b in sorted(bridges, key=lambda x: -x.strength()):
    print(f"  {b.concept}")
    print(f"    Vaults: {b.vaults}")
    print(f"    Strength: {b.strength():.0f}")
    print()

## Visualize

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Build visualization graph
G = nx.Graph()

bridge_concepts = {b.concept for b in bridges}

for eid, edge in merged.hyperedges.items():
    vault = edge.metadata.get('vault', 'unknown')
    G.add_node(edge.name, node_type='doc', vault=vault)
    
    for member in edge.members:
        if member in bridge_concepts:
            G.add_node(member, node_type='bridge')
            G.add_edge(edge.name, member)

# Draw
fig, ax = plt.subplots(figsize=(12, 8))
pos = nx.spring_layout(G, k=2, seed=42)

# Color by type
doc_nodes = [n for n, d in G.nodes(data=True) if d.get('node_type') == 'doc']
bridge_nodes = [n for n, d in G.nodes(data=True) if d.get('node_type') == 'bridge']

vault_colors = {'Philosophy': '#e94560', 'AI_Research': '#0f3460'}
doc_colors = [vault_colors.get(G.nodes[n].get('vault'), '#888888') for n in doc_nodes]

nx.draw_networkx_edges(G, pos, alpha=0.3)
nx.draw_networkx_nodes(G, pos, nodelist=doc_nodes, node_color=doc_colors, node_size=300, alpha=0.8)
nx.draw_networkx_nodes(G, pos, nodelist=bridge_nodes, node_color='#f9a825', node_size=500, alpha=0.9)
nx.draw_networkx_labels(G, pos, font_size=8)

plt.title('Cross-Vault Bridges', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

## Export for External Tools

In [ ]:
# Export to various formats
from visualization_export import save_graphml, save_dot, save_json

save_graphml(merged, 'vault_graph.graphml')
save_json(merged, 'vault_graph.json')

# Cytoscape format (for web visualization)
cyto_json = to_cytoscape_json(merged)
with open('vault_cytoscape.json', 'w') as f:
    f.write(cyto_json)

print("Exported:")
print("  - vault_graph.graphml (Gephi, yEd)")
print("  - vault_graph.json (programmatic access)")
print("  - vault_cytoscape.json (Cytoscape.js web viz)")

# Download
from google.colab import files
files.download('vault_graph.graphml')

## Next Steps

- **Temporal tracking**: Use `TemporalHypergraph` to version changes over time
- **MCP integration**: Connect to Claude or other LLM tools
- **Full documentation**: [GitHub repo](https://github.com/dabirdwell/structured-emergence/tree/main/tools/temporal-hypergraph)

---
*Part of the [Structured Emergence](https://github.com/dabirdwell/structured-emergence) research project.*